In [1]:
import numpy as np

import d3m
from d3m.container import pandas as pd
from d3m.container.dataset import D3MDatasetLoader
from d3m.metadata import hyperparams
import d3m.container

import os

from jhu_primitives.utils import file_path_conversion
from jhu_primitives.utils.util import data_file_uri

from sklearn.decomposition import TruncatedSVD as tSVD

import jhu_primitives as jhu

from jhu_primitives.utils import file_path_conversion
from jhu_primitives.utils.util import data_file_uri

from sklearn.decomposition import TruncatedSVD as tSVD
import networkx as nx

# abs_file_path = os.path.abspath(os.getcwd())

# dataset_uri_TRUTH = data_file_uri(abs_file_path, uri = "file", datasetDoc = True, dataset_type = "")
# data_TRUTH = D3MDatasetLoader().load(dataset_uri = dataset_uri_TRUTH)

# dataset_uri_TRAIN = data_file_uri(abs_file_path, uri = "file", datasetDoc = True, dataset_type = "TRAIN")
# data_TRAIN = D3MDatasetLoader().load(dataset_uri = dataset_uri_TRAIN)

# dataset_uri_TEST = data_file_uri(abs_file_path, uri = "file", datasetDoc = True, dataset_type = "TEST")
# data_TEST = D3MDatasetLoader().load(dataset_uri = dataset_uri_TEST)

# pickle.dump(data_TRUTH, open('umls_truth.pkl', 'wb'))
# pickle.dump(data_TRAIN, open('umls_train.pkl', 'wb'))
# pickle.dump(data_TEST, open('umls_test.pkl', 'wb'))

import _pickle as pickle

truth = pickle.load(open('umls_truth.pkl', 'rb'))
train = pickle.load(open('umls_train.pkl', 'rb'))
test = pickle.load(open('umls_test.pkl', 'rb'))

# 59_umls

d3m.primitives.link_prediction.data_conversion.JHU: Primitive's Python path does not adhere to d3m.primitives namespace specification (see https://gitlab.com/datadrivendiscovery/d3m/issues/3). Reason: must have a known primitive name segment.
d3m.primitives.link_prediction.data_conversion.JHU: Primitive is not providing a description through its docstring.
d3m.primitives.jhu_primitives.LinkPredictionRankClassifier: Primitive's Python path does not adhere to d3m.primitives namespace specification (see https://gitlab.com/datadrivendiscovery/d3m/issues/3). Reason: must have 5 segments. This API will be made mandatory after January 2019 and this warning will become an exception.
d3m.primitives.jhu_primitives.LinkPredictionRankClassifier: Primitive is not providing a description through its docstring.


In [2]:
# 59_umls LinkPrediction pipeline

In [3]:
hp_lpgr = jhu.link_pred_graph_reader.link_pred_graph_reader.Hyperparams().defaults()
hp_ase = jhu.ase.ase.Hyperparams({'which_elbow': 1, 'max_dimension': 2, 'use_attributes': False})
hp_lprc = jhu.link_pred_rc.link_pred_rc.Hyperparams().defaults()

GR = jhu.LinkPredictionGraphReader(hyperparams=hp_lpgr)
ASE = jhu.AdjacencySpectralEmbedding(hyperparams=hp_ase)
RC = jhu.LinkPredictionRankClassifier(hyperparams = hp_lprc)

gr_train = GR.produce(inputs = train).value
ase_train = ASE.produce(inputs = gr_train).value
RC.set_training_data(inputs = ase_train)
RC.fit()

gr_test = GR.produce(inputs = test).value
ase_test = ASE.produce(inputs = gr_test).value
predictions = RC.produce(inputs = ase_test).value

truth_labels = np.array(truth['learningData']['linkExists'])[np.array(predictions['d3mIndex']).astype(int)]
preds = np.array(predictions['linkExists'])
np.sum(preds.astype(str) == truth_labels)/len(preds)

0.702258726899384